In [2]:
import pandas as pd
import json

In [5]:
import pandas as pd
import json

df = pd.read_csv('../data/Fine-tuning Data_preprocess.csv').drop(columns="Unnamed: 0")

# 딕셔너리로 변환 후 JSON 저장
data = df.to_dict(orient='records')

with open('../data/Fine-tuning Data_preprocess.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

In [ ]:
# JSON 파일 제대로 로드되는지 확인
try:
    with open('../data/Fine-tuning Data_preprocess.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    print("JSON 로드 성공!")
    print(f"데이터 개수: {len(data)}")
    print(f"첫 번째 항목: {data[0]}")
except Exception as e:
    print(f"에러 발생: {e}")

JSON 로드 성공!
데이터 개수: 100
첫 번째 항목: {'text': 'GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹 관리에서 나아가 그룹 내 학습 내용을 복습할 수 있는 기능을 제공하여 학습 효과를 극대화하는 것을 목표로 한다. 사용자들은 회원가입과 로그인을 통해 서비스를 이용할 수 있으며, 아이디 및 비밀번호 찾기 기능으로 편의성을 높였다. 사용자는 새로운 스터디 그룹을 생성하거나 기존 그룹을 검색해 가입할 수 있으며, 그룹 내에서는 퀴즈를 생성하고 참여하여 학습한 내용을 효과적으로 반복 학습할 수 있다. 또한 공지사항 게시판을 통해 그룹 내 전달사항을 공유할 수 있고, 과제 게시판은 체크리스트 형태로 구성되어 과제 수행 여부를 명확히 기록할 수 있다. 이러한 기능들은 사용자 간의 소통을 강화하고 학습 동기를 부여하는 데 기여한다. 서비스는 Java Servlet과 JSP 기반으로 구현되었으며, HTML, CSS, JavaScript를 통해 사용자 친화적인 인터페이스를 제공한다. 데이터베이스는 MySQL을 사용하여 안정적인 데이터 저장과 처리를 가능하게 하며, 전반적인 시스템은 학습 관리에 최적화된 구조로 설계되었다.', 'contents': "{'project_summary': {'title': 'GQ 서비스', 'category': '웹 플랫폼', 'target_users': ['학생', '학습자', '스터디 그룹 운영자'], 'main_purpose': '학습 효과 극대화를 위한 스터디 그룹 운영 지원', 'key_features': [{'feature': '회원가입 및 로그인', 'description': '사용자 편의성을 위한 아이디 및 비밀번호 찾기 기능 포함'}, {'feature': '스터디 그룹 생성 및 가입', 'description': '새로운 그룹 생성 및 기존 그룹 검색 기능 제공'}, {'feature': '퀴즈 생성 및 참여', 'description': '학습 내용을 효과적으로 반복 학

In [9]:

import pandas as pd
import json
from datasets import Dataset

# CSV 파일 읽기
df = pd.read_csv('../data/Fine-tuning Data_preprocess.csv', encoding='utf-8')

# Unnamed 컬럼 제거 (있는 경우)
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns='Unnamed: 0')

def create_finetuning_dataset(df):
    """
    CSV 데이터를 파인튜닝 형식으로 변환
    text 컬럼 -> instruction
    나머지 컬럼들 -> output으로 결합
    """
    finetuning_data = []
    
    for _, row in df.iterrows():
        # text 컬럼을 instruction으로 사용
        instruction = row['text']
        
        # 나머지 컬럼들을 output으로 결합
        output_parts = []
        
        if pd.notna(row['contents']) and row['contents']:
            output_parts.append(f"**프로젝트 상세 정보:**\n{row['contents']}")
        
        if pd.notna(row['relationships_data']) and row['relationships_data']:
            output_parts.append(f"**관계 데이터:**\n{row['relationships_data']}")
        
        if pd.notna(row['ERD_data']) and row['ERD_data']:
            output_parts.append(f"**ERD 데이터:**\n{row['ERD_data']}")
        
        if pd.notna(row['API_specs_data']) and row['API_specs_data']:
            output_parts.append(f"**API 명세 데이터:**\n{row['API_specs_data']}")
        
        output = '\n\n'.join(output_parts)
        
        finetuning_data.append({
            'instruction': instruction,
            'output': output
        })
    
    return finetuning_data

# 파인튜닝 데이터셋 생성
finetuning_dataset = create_finetuning_dataset(df)

# JSON 파일로 저장
with open('../data/finetuning_dataset.json', 'w', encoding='utf-8') as f:
    json.dump(finetuning_dataset, f, ensure_ascii=False, indent=2)

print(f"파인튜닝 데이터셋 생성 완료!")
print(f"총 {len(finetuning_dataset)}개의 데이터")

# 첫 번째 샘플 확인
print("\n첫 번째 샘플:")
print("Instruction:", finetuning_dataset[0]['instruction'][:150] + "...")
print("Output 길이:", len(finetuning_dataset[0]['output']))


파인튜닝 데이터셋 생성 완료!
총 100개의 데이터

첫 번째 샘플:
Instruction: GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹 관리에서 나아가 그룹 내 학습 내용을 복습할 수 있는 기능을 제공하여 학습 효과를 극대화하는 것을 목표로 한다. 사용자들은 회원가입과 로그인을 통해 서비스를 이용할 수 있으며, 아이디 ...
Output 길이: 13819
